In [1]:
import keras
from keras.models import Sequential
from PIL import Image
import numpy as np

Using TensorFlow backend.


In [2]:
import pandas as pd

In [3]:
import keras.backend as K
K.set_image_data_format('channels_last')

In [19]:
base_model = keras.applications.vgg16.VGG16(weights = 'imagenet',  include_top = False, input_shape=(64,64,3))

In [20]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [21]:
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import Adam

_ = base_model.get_layer("block5_conv3").output
_ = GlobalAveragePooling2D()(_)
_ = Dense(512, activation='relu')(_)

# 3 個輸出
predictions = Dense(3, activation='softmax')(_)

# 這是我們的 model
model = Model(inputs=base_model.input, outputs=predictions)

for n in ["block5_conv1", "block5_conv2", "block5_conv3"]:
    model.get_layer(n).trainable = True
    
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=["accuracy"])

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [9]:
train_data = np.load('/home/Kaggle_Cervical_Cancer_Screening/Rory/Dev-ipynb/resize_data/ALL_TRAIN_NOSEG_64.npy')

In [24]:
test_data = np.load('/home/Kaggle_Cervical_Cancer_Screening/Rory/Dev-ipynb/resize_data/ALL_TEST_NOSEG_64.npy')

In [25]:
train = pd.read_csv('/home/Kaggle_Cervical_Cancer_Screening/Rory/Dev-ipynb/resize_data/TRAIN.csv')
test = pd.read_csv('/home/Kaggle_Cervical_Cancer_Screening/Rory/Dev-ipynb/resize_data/TEST.csv')

In [26]:
test_id = test.image.values

In [27]:
le = LabelEncoder()
train_target = le.fit_transform(train['type'].values)

x_train, x_val_train, y_train, y_val_train =\
train_test_split(train_data, train_target, test_size = 0.4, random_state=17)

In [28]:
num_classes = 3
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val_train = keras.utils.to_categorical(y_val_train, num_classes)

In [29]:
x_train = x_train.astype('float32')
x_val_train = x_val_train.astype('float32')

x_train /= 255
x_val_train /= 255

x_test = test_data.astype('float32')
x_test /= 255

In [30]:
from keras.preprocessing.image import ImageDataGenerator

In [31]:
# shift = 0.2
datagen = ImageDataGenerator(rotation_range = 180,\
                             horizontal_flip = True,\
                             vertical_flip = True,\
                             #width_shift_range=shift,\
                             #height_shift_range=shift,\
                             data_format = "channels_last")
datagen.fit(x_train)

In [32]:
model.fit_generator(datagen.flow(x_train, y_train,\
                                 batch_size = 128, shuffle = True),\
                    epochs = 12,\
                    steps_per_epoch = 500,\
                    verbose = 1,\
                    validation_data = (x_val_train, y_val_train))

Epoch 1/12
500/500 [==============================] - 118s - loss: 0.9001 - acc: 0.5600 - val_loss: 0.8529 - val_acc: 0.5921

In [33]:
x_test.shape

(512, 64, 64, 3)

In [34]:
pred = model.predict(x_test)

In [35]:
pred.shape

(512, 3)

In [36]:
df = pd.DataFrame(pred, columns=['Type_1','Type_2','Type_3'])

In [37]:
df['image_name'] = test_id

In [38]:
df = df[['image_name','Type_1','Type_2','Type_3']]

In [39]:
df

,image_name,Type_1,Type_2,Type_3
0,477.jpg,4.996730e-01,5.002996e-01,2.750324e-05
1,178.jpg,1.145197e-06,2.974700e-05,9.999691e-01
2,335.jpg,2.781465e-03,7.233045e-01,2.739141e-01
3,324.jpg,4.085184e-04,9.987413e-01,8.502507e-04
4,98.jpg,5.006029e-02,9.494617e-01,4.779883e-04
5,184.jpg,9.536579e-08,1.308036e-06,9.999986e-01
6,330.jpg,9.582498e-01,4.158361e-02,1.664994e-04
7,170.jpg,1.078152e-04,9.973841e-01,2.507980e-03
8,163.jpg,8.245084e-03,9.914375e-01,3.174098e-04
9,344.jpg,9.986015e-01,1.382072e-03,1.645589e-05


In [41]:
df.to_csv('/home/Kaggle_Cervical_Cancer_Screening/submission/submission_LB_vgg16_2.csv', index=False)

In [42]:
pd.read_csv('/home/Kaggle_Cervical_Cancer_Screening/submission/submission_LB_vgg16_2.csv').head()

,image_name,Type_1,Type_2,Type_3
0,477.jpg,0.499673,0.500300,0.000028
1,178.jpg,0.000001,0.000030,0.999969
2,335.jpg,0.002781,0.723305,0.273914
3,324.jpg,0.000409,0.998741,0.000850
4,98.jpg,0.050060,0.949462,0.000478
